# AWS IoT Connectivity and Security Basics


To start the workshop we are going to create a Thing inside AWS IoT. A Thing should represent any phisycal thing in world: button, a raspberry pi, a lamp or even your own laptop can be a thing. We are going to identify this thing by a name starting with MyHealthThing followed by six numbers.

In [1]:
%store -r home
home

'/home/ec2-user'

Create a Thing, a unique name helps to make references more clear.
Any computer can be a thing, lets use this notebook instance as our thing to begin.

In [2]:
%store -r unique
unique

'ehw110650'

In [3]:
thing_name = "bp_monitor_{}".format(unique)
%store thing_name
thing_name

Stored 'thing_name' (str)


'bp_monitor_ehw110650'

In [4]:
! aws iot create-thing --thing-name {thing_name}

{
    "thingName": "bp_monitor_ehw110650",
    "thingArn": "arn:aws:iot:us-east-1:030555009967:thing/bp_monitor_ehw110650",
    "thingId": "c87f5c5b-9e76-44ee-92d0-c0401ae596cc"
}


# Authentication and Authorization


Create a certificate and keys to authenticate your thing. 

In [5]:
! mkdir -p "{home}/aws_iot"

certificate_file = "{}/aws_iot/certificate{}.pem".format(home,unique)
public_key = "{}/aws_iot/public-key{}.pem".format(home,unique)
private_key = "{}/aws_iot/private-key{}.pem".format(home,unique)

%store certificate_file
%store public_key
%store private_key

Stored 'certificate_file' (str)
Stored 'public_key' (str)
Stored 'private_key' (str)


In [6]:
certificateArn = ! aws iot create-keys-and-certificate \
  --set-as-active \
  --certificate-pem-outfile {certificate_file} \
  --public-key-outfile {public_key} \
  --private-key-outfile {private_key} \
  --query certificateArn \
  --output text
certificate_arn = certificateArn.s
certificate_arn

'arn:aws:iot:us-east-1:030555009967:cert/9107aa894302b363b8bbb4c79c066ee7a2c5b0acc5065a8c08d1a18cc5609cc8'

Download the root Certification Authority used by AWS IoT


In [7]:
ca_file = "{}/aws_iot/ca{}.pem".format(home,unique)
%store ca_file
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {ca_file}

Stored 'ca_file' (str)
--2018-11-01 11:06:59--  https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem
Resolving www.symantec.com (www.symantec.com)... 104.110.202.116, 2600:1408:10:1b4::145b, 2600:1408:10:199::145b
Connecting to www.symantec.com (www.symantec.com)|104.110.202.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1758 (1.7K) [text/plain]
Saving to: ‘/home/ec2-user/aws_iot/caehw110650.pem’

/home/ec2-user/aws_ 100%[===================>]   1.72K  --.-KB/s    in 0s      

2018-11-01 11:06:59 (375 MB/s) - ‘/home/ec2-user/aws_iot/caehw110650.pem’ saved [1758/1758]



Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [8]:
! printf "Private Key {private_key} " && [ -f {private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {public_key} " && [ -f {public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {certificate_file} " && [ -f {certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {ca_file} " && [ -f {ca_file} ] && printf "found" || printf "not found"

Private Key /home/ec2-user/aws_iot/private-keyehw110650.pem found
Public Key /home/ec2-user/aws_iot/public-keyehw110650.pem found
Certificate /home/ec2-user/aws_iot/certificateehw110650.pem found
Certification Authority /home/ec2-user/aws_iot/caehw110650.pem found

Create an IAM authorization policy stating what is going to be allowed.

In [9]:
#TODO: Use a more restrictive policy
policy_document = '''{
  "Version": "2012-10-17",
  "Statement": [{
    "Effect": "Allow",
    "Action": "iot:*",
    "Resource": "*"}]
}'''
policy_document

'{\n  "Version": "2012-10-17",\n  "Statement": [{\n    "Effect": "Allow",\n    "Action": "iot:*",\n    "Resource": "*"}]\n}'

In [10]:
policy_name = "NotebooksPolicy{}".format(unique)
policyArn = ! aws iot create-policy \
    --policy-name {policy_name} \
    --policy-document '{policy_document}' \
    --query policyArn \
    --output text
policyArn.s

'arn:aws:iot:us-east-1:030555009967:policy/NotebooksPolicyehw110650'

Attach certificate to thing and policy

In [11]:
! aws iot attach-policy \
    --policy-name {policy_name} \
    --target {certificate_arn}

In [12]:
! aws iot attach-thing-principal \
  --thing-name {thing_name} \
  --principal {certificate_arn}

# Connecting to AWS IoT

AWS IoT provides account-specific endpoints for your things to connect:

In [13]:
endpoint_address = !aws iot describe-endpoint --query endpointAddress --output=text
endpoint_address = endpoint_address.s
%store endpoint_address
endpoint_address

Stored 'endpoint_address' (str)


'a3p88ooj9y6ruv.iot.us-east-1.amazonaws.com'

Ready to go! Create your MQTT client instance:

In [14]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient{}".format(unique)
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt

# Receiving and Sending Messages

Connect to the service to send and receive messages:

In [15]:
mqtt.connect()

True

Subscribe to a topic:

In [16]:
topic_name ="myhealthcare/data"
%store topic_name
subscribe_qos = 1
def custom_callback(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    print("\n--------------\n")
    
mqtt.subscribe(topic_name, subscribe_qos, custom_callback)

Stored 'topic_name' (str)


True

Publish to a topic:

In [17]:
data = {"id": "fa91d2de-24a1-4052-8b58-4f2ab743f20e",
        "name": "Clara Oswald", 
        "systolic": 120, 
        "diastolic": 80, 
        "time_stamp": 1541059348245}
import json
payload = json.dumps(data, indent=4)
%store payload
print(payload)

Stored 'payload' (str)
{
    "id": "fa91d2de-24a1-4052-8b58-4f2ab743f20e",
    "name": "Clara Oswald",
    "systolic": 120,
    "diastolic": 80,
    "time_stamp": 1541059348245
}


In [18]:
publish_qos = 0
mqtt.publish(topic_name, payload , publish_qos)

True

In [19]:
#Wait for it
import time
time.sleep(1)


Received a new message: 

{
    "id": "fa91d2de-24a1-4052-8b58-4f2ab743f20e",
    "name": "Clara Oswald",
    "systolic": 120,
    "diastolic": 80,
    "time_stamp": 1541059348245
}

--------------



The subscribe_qos and publish_qos arguments refer to the quality of service parameter of the MQTT protocol. Selecting the QoS level is a tradeoff between delivery guarantee and performance, see the following resources for more infomation:

https://www.hivemq.com/blog/mqtt-essentials-part-6-mqtt-quality-of-service-levels

https://docs.aws.amazon.com//iot/latest/developerguide/protocols.html

All working fine, done for now, disconnect.

In [20]:
mqtt.disconnect()

True

Although sending and receiving messages is quite simple, device state management can be complex. For example, consider an IoT lightbulb controled by a mobile app, sending on/off requests and displaying the current state. If any of the mobiles or the lightbulb is temporarilly disconnected, messages will be lost, requiring state reconciliation to display  the light correctly. For more sophisticated devices, state merging and management can consume significant development efforts.

AWS IoT Core can store and manage the "device shadow" on the cloud and calculate the difference between the reported and desired device state. Let's see how that works in the [Managing Device State](aws-iot-shadow.ipynb) notebook.